# Denison CS181/DA210 SW Lab #14 - Step 1

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

In [ ]:
import os
import os.path
import sys
import importlib
import json
import requests

if os.path.isdir(os.path.join("../../..", "modules")):
    module_dir = os.path.join("../../..", "modules")
else:
    module_dir = os.path.join("../..", "modules")

module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

import util
importlib.reload(util)

---

## Part A: Endpoints (via GitHub API)

API providers generally define a set of _endpoints_, which utilize the resource-tree structure of the resource path in an HTTP request.

We'll first consider the [GitHub API](https://docs.github.com/en/rest).  The provider, GitHub, defines a _root endpoint_ that serves as a "meta" endpoint, providing data about the other endpoints available.  This data is given as a JSON object.

In [ ]:
# Access the root endpoint of the GitHub API
resource_path = "/"
location = "api.github.com"
url = util.buildURL(resource_path, location)

response = requests.get(url)
assert response.status_code == 200

# The root endpoint provides a JSON-formatted result
# of the top-level endpoints available in the GitHub API
endpoints = response.json()
util.print_data(endpoints, nlines=10)

The results from the GitHub API's root endpoint include specific endpoints for interacting with, among other things:

- organizations:

  - to list repositories of a particular organization and
  - to get detailed information about an organization.

- repositories:

  - to list public repositories.

- users/owners:

  - to retrieve basic information about the user,
  - to find the set of users who are followers of a user,
  - to find the set of users this user is following, and
  - to retrieve information about a particular repository of this user.

As an example, we'll look at the `/events` endpoint.

In [ ]:
# Access the non-root /events endpoint of the GitHub API
resource_path = "/events"
url = util.buildURL(resource_path, location)

response = requests.get(url)
assert response.status_code == 200

# The /events endpoint provides a JSON-formatted list of dictionaries,
# with each dictionary representing one event
event_list = response.json()
print("Number of events received:", len(event_list))

util.print_data(event_list, depth=2, nchild=7, nlines=25)

---

## Part B: Path Parameters

In the previous section, we considered the `/events` endpoint, which did not make use of parameters.  Therefore, we could also view the results in a browswer: http://api.github.com/events.

Some APIs use the _endpoint-path_ portion of the _resource-path_ in a request to provide parameters as steps within the path.  An example of this is the endpoint to query information about a specific organization from the GitHub API.

Such a request is shown as:
```
GET /orgs/:org
```
where `:org` specifies a parameter that must be inserted.  Note that some API documentation will use `{variable}` or `<<variable>>` instead of `:variable` to identify parameters that must be provided.

You can see more about this endpoint in the [GitHub API documentation](https://docs.github.com/en/rest/orgs/orgs#get-an-organization).

We'll try it out for the `microsoft` organization.

In [ ]:
# Use the endpoint-path portion of the resource-path to encode parameters
org = "microsoft"
resource_path = f"/orgs/{org}" # using format-string instead of s.format()
url = util.buildURL(resource_path, location)

response = requests.get(url)
assert response.status_code == 200

# The /orgs endpoint provides JSON-formatted info about a given org
data = response.json()
util.print_data(data, depth=3, nchild=7, width=80)

Similarly, we can view events related to a specific repository belonging to a specific organization, using the endpoint:
```
GET /repos/:owner/:repo/events
```

One public Microsoft repository is [`vscode`](https://github.com/microsoft/vscode), the repository housing the code for [Visual Studio Code](https://code.visualstudio.com/).  We'll again use path parameters to perform a `GET` request using this endpoint.

In [ ]:
# Use the endpoint-path portion of the resource-path to encode parameters
org = "microsoft"
repo = "vscode"
resource_path = f"/repos/{org}/{repo}/events" # using format-string
url = util.buildURL(resource_path, location)

response = requests.get(url)
assert response.status_code == 200

# The /repos/:owner/:repo/events endpoint provides a list of
# JSON-formatted events for a given repository belonging to a given org
data = response.json()
util.print_data(data, depth=3, nchild=7, nlines=30, width=80)

**Q1:** Now, try out another GitHub API endpoint that uses path parameters.  For example, you could [list repository contributors](https://docs.github.com/en/rest/repos/repos#list-repository-contributors), [list repository issues](https://docs.github.com/en/rest/issues/issues#list-repository-issues), or [list your own public repositories](https://docs.github.com/en/rest/repos/repos#list-repositories-for-a-user).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

> You've reached the first checkpoint in the lab.  Make sure to have it signed off by the instructor or TA.
>
> Checkpoint 1: Look at the documentation for the GitHub API endpoint you tried out.  What are the possible status codes your request can return?

---

## Part C: Authenticating API Key

Although GitHub has endpoints that can illustrate the remaining concepts we'll explore, it can be helpful to see multiple examples.  For this, we'll use The Movie Database (TMDB).

First, you'll need to make a free TMDB account to work with its API: [https://www.themoviedb.org/signup](https://www.themoviedb.org/signup).

Here are some helpful links:

- TMDB Terms of Use: [https://www.themoviedb.org/documentation/api/terms-of-use](https://www.themoviedb.org/documentation/api/terms-of-use)
- TMDB API Getting Started page: [https://developers.themoviedb.org/3/getting-started/introduction](https://developers.themoviedb.org/3/getting-started/introduction)
- TMDB API Documentation: [https://www.themoviedb.org/documentation/api](https://www.themoviedb.org/documentation/api)

Some APIs require an API key, which is obtained by an application developer, and identifies the application the API.  Your API key is like a password -- you should not share it with anyone!

Before continuing, register an "application" with TMDB to get an API key: [https://www.themoviedb.org/settings/api](https://www.themoviedb.org/settings/api).

Note that you do not need to give too much information.  For example, you could reasonably use Denison's campus address for the address.  Here is an example:

```
    Application Type: Personal

    Application Name: Test Application
    Application URL: localhost
    Application Summary: Test application to try out TMDB API

    Address: 100 West College Street, Granville, Ohio 43023
```

You should treat your API key like a password -- it unqiuely identifies you.  We'll store it locally in a `creds.json` file.  Open [creds.json](`creds.json`) and paste your "API Key (v3 auth) from https://www.themoviedb.org/settings/api as the value for the key `"apikey"`.  Your file should look something like this (note that the API key below is not valid, but it's the same length yours should be):

In [ ]:
{
    "tmdb": {
        "protocol": "https",
        "location": "api.themoviedb.org",
        "apikey": "64cefb3e82db8d373423a7bd0aa8956d"
    }
}

**Q2:** Perform an API request given your API key.  Note that we can use another utility function to parse the JSON-formatted `creds.json`, returning the dictionary for `"tmdb"` information.

Write code to access this endpoint: `https://api.themoviedb.org/3/tv/popular?api_key={apikey}`.

In [ ]:
# Use a util function to read the API key from creds.json
tmdb_creds = util.read_creds("tmdb", ".", "creds.json")
apikey = tmdb_creds["apikey"]

# YOUR CODE HERE
raise NotImplementedError()

> You've reached the second checkpoint in the lab.  Make sure to have it signed off by the instructor or TA.
>
> Checkpoint 2: What is the type of the result for this request?  How many results does it return?  From looking at the [documentation for this endpoint](https://developers.themoviedb.org/3/tv/get-popular-tv-shows), what are the possible response status codes?

---

## Part D: Query Parameters

In addition to the API key, we can provide other parameters as part of the _query-string_ portion of the _resource-path_ in a `GET` request.  For example, the `/search` endpoint of the TMDB API allows us to provide the `query` string to search for movies matching that query.

In [ ]:
# Build the URL
resource_path = "/3/search/movie"
location = "api.themoviedb.org"
url = util.buildURL(resource_path, location)

# Use a util function to read the API key from creds.json
tmdb_creds = util.read_creds("tmdb", ".", "creds.json")
apikey = tmdb_creds["apikey"]

query_params = {"query": "Star Wars",
                "api_key": apikey}

# Use the query parameters within the request
response = requests.get(url, params=query_params)
print(response.status_code)
assert response.status_code == 200

# Look at how the path URL was encoded (e.g., the space as +)
print(response.request.path_url[:46] + "...")

# View the resulting JSON
search_results = response.json()
util.print_data(search_results, depth=2, nchild=9, nlines=30, width=80)

---

## Part E: Header Parameters

The final place we may supply parameters to a `GET` request is within the request headers.  For example, even though TMDB always returns JSON-formatted data, some other APIs may allow the client to specify the desired data format, e.g., `"Accept": "application/json"` or `"Accept": "text/xml"`.

Looking at the TMDB API documentation, you would find the headers are required for `POST` and `DELETE` requests, as well as `GET` requests related to a specific account; these headers have additional authentication information (discussed in Chatper 24).

For now, we can still create a headers dictionary and provide it for our `GET` requests, even if it may be ignored.

In [ ]:
# Build the URL
resource_path = "/3/search/movie"
location = "api.themoviedb.org"
url = util.buildURL(resource_path, location)

# Use a util function to read the API key from creds.json
tmdb_creds = util.read_creds("tmdb", ".", "creds.json")
apikey = tmdb_creds["apikey"]

query_params = {"query": "Star Wars",
                "api_key": apikey}
header_params = {"Content-Type": "application/json",
                 "Accept-Encoding": "gzip, deflate"}

# Use the query and header parameters within the request
response = requests.get(url, params=query_params, headers=header_params)
assert response.status_code == 200

# View the headers of the request
request = response.request
util.print_headers(request.headers)

Note that our headers may be ignored.  For instance, TMDB will only provide JSON-formatted data, so if we give a different `Content-Type`, it still provides valid JSON.

In [ ]:
# Repeat the previous request, but specify a different content type
# (which will be ignored by the API)
header_params = {"Content-Type": "text/html",
                 "Accept-Encoding": "gzip, deflate"}

# Use the query and header parameters within the request
response = requests.get(url, params=query_params, headers=header_params)
assert response.status_code == 200

# View the headers of the request
request = response.request
util.print_headers(request.headers)

In [ ]:
# Try to parse the result as JSON
data = response.json()
util.print_data(search_results, depth=2, nchild=9, nlines=10, width=80)

However, some APIs are more restrictive than general web resources, so it is not a good idea to use unsupported headers.  For instance, the TMDB API does not support `"Transfer-Encoding": "chunked"`, and adding this header results in a `501: Not Implemented` status code for the response.

In [ ]:
# Repeat the previous request, but add an unsupported header
header_params = {"Content-Type": "text/html",
                 "Accept-Encoding": "gzip, deflate",
                 "Transfer-Encoding": "chunked"}

# Use the query and header parameters within the request
response = requests.get(url, params=query_params, headers=header_params)
print("Status code:", response.status_code) # 501: Not Implemented

---

## Part F: `POST` and `POST` Body

For our final example, we build a `POST` request to rate a movie using the TMDB API.  If we look at the [documentation for this endpoint](https://developers.themoviedb.org/3/movies/rate-movie), we'll see that we need the following:

- Path parameter: `movie_id` (integer) - the movie to rate
- Header: `Content-Type` (string) - content type, defaults to `"application/json;charset=utf-8"`
- Query String: `api_key` (string) - your API key
- Request Body: `value` (number) - the rating

The `POST` request will not succeed unless we provide a guest session ID (this is separate from the user's API key).  First, we retrieve a guest session ID:

In [ ]:
# Build the URL
resource_path = "/3/authentication/guest_session/new"
location = "api.themoviedb.org"
url = util.buildURL(resource_path, location)

# Use a util function to read the API key from creds.json
tmdb_creds = util.read_creds("tmdb", ".", "creds.json")
apikey = tmdb_creds["apikey"]

query_params = {"api_key": apikey}

# Perform a GET request to receive a guest session ID
response = requests.get(url, params=query_params)
assert response.status_code == 200

results = response.json()
util.print_data(results, depth=2, nchild=3)

Copy the guest session ID you receive into the code cell below.  Now we can execute the `POST` request.  Note the provided body, as well as the expected status code for the response.

In [ ]:
# Use the endpoint-path portion of the resource-path to encode parameters
movie_id = 181812
resource_path = f"/3/movie/{movie_id}/rating"
location = "api.themoviedb.org"
url = util.buildURL(resource_path, location)

# Use a util function to read the API key from creds.json
tmdb_creds = util.read_creds("tmdb", ".", "creds.json")
apikey = tmdb_creds["apikey"]

# TODO: use the result from the previous cell's output
guest_session_id = "# TODO"
query_params = {"api_key": apikey,
                "guest_session_id": guest_session_id}
header_params = {"Content-Type": "application/json;charset=utf-8"}

# Build the body
body = {"value": 9.5}
bodystr = json.dumps(body)

# Make the POST request
response = requests.post(url,
                         params=query_params,
                         headers=header_params,
                         data=bodystr)
print(response.status_code)
assert response.status_code == 201

util.print_data(response.json())

In [ ]:
print(bodystr)

In [ ]:
print(response.request.body)

> You've reached the third checkpoint in the lab.  Make sure to have it signed off by the instructor or TA.
>
> Checkpoint 3: Look at the response information printed/asserted for this example.  What does the status code mean?  What is the data type of the request body?  How do you know?

---

## Part G: Try It Out Yourself!

**Q3:** Write a function 
```
    getGenreDict(my_api_key)
``` 
that returns a Python dictionary mapping **Movie** genre names to id numbers, e.g.,
```
    {'Action': 28,
     'Adventure': 12,
      ...}
```

Your function should only make a single call to `requests.get()`, with the appropriate [endpoint path](https://developers.themoviedb.org/3/genres/get-movie-list) for TMDB, and with the given API key.

Return `None` if something goes wrong.  Note that this is not asking for the entire JSON result, but a dictionary mapping genre names to id numbers.

You may want to write this as a global cell to get it working and to examine the results, and then convert it into a function.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Debugging cell
d = getGenreDict(apikey)
print(d)

In [ ]:
# Testing cell
d = getGenreDict(apikey)
assert len(d) == 19
assert d["Action"] == 28
assert d["Fantasy"] == 14
assert d["Music"] == 10402

**Q4:** Write a function 
```
    search_person_id(my_api_key, name)
``` 
that uses the `/search/person` endpoint ([https://developers.themoviedb.org/3/search/search-people](https://developers.themoviedb.org/3/search/search-people)) to conduct a search for the given `name`, and returns the `id` of the first entry in the `results` list. This should use only one call to `requests.get()`.

Return `None` if something goes wrong, and return `-1` if there were no results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Debugging cell
print(search_person_id(apikey, "Karen Gillan"))

In [ ]:
# Testing cell
assert search_person_id(apikey, "Karen Gillan") == 543261
assert search_person_id(apikey, "Bradley Cooper") == 51329

assert search_person_id(apikey, "Lulu Amert") == None # no results for a cat
assert search_person_id(apikey+'z', 'Bill Murray') == None # invalid API key

> You've reached the fourth (and final) checkpoint in the lab.  Make sure to have it signed off by the instructor or TA.
>
> Checkpoint 4: For each of the previous two questions, what parameters did you need (that is, path parameters, query parameters, header parameters)?

---

---

## Part H

How much time (in minutes/hours) did you spend on this lab outside of class?

YOUR ANSWER HERE